# 🔒 PGD Adversarial Attack on MNIST Model
This notebook demonstrates how to test a simple MNIST model against Projected Gradient Descent (PGD) attacks using `cleverhans`.

In [1]:
!pip install torch torchvision pandas cleverhans pytest

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.3/92.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 11.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu1

In [2]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import numpy as np
import pandas as pd
from cleverhans.torch.attacks.projected_gradient_descent import projected_gradient_descent

In [3]:
# Load MNIST test set
transform = transforms.Compose([transforms.ToTensor()])
test_dataset = datasets.MNIST(root="./data", train=False, transform=transform, download=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=True)

100%|██████████| 9.91M/9.91M [00:00<00:00, 45.5MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.86MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 12.3MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 4.11MB/s]


In [4]:
# Define a simple model architecture
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc = nn.Linear(28 * 28, 10)

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        return self.fc(x)

!wget https://github.com/tdvan1804/PracticeRepo/raw/main/simple_mnist_model.pth

# Load pre-trained model (replace with path if needed)
model = SimpleNN()
model.load_state_dict(torch.load("simple_mnist_model.pth", map_location=torch.device('cpu')))
model.eval()

FileNotFoundError: [Errno 2] No such file or directory: 'simple_mnist_model.pth'

In [ ]:
# Test PGD attack at different epsilon values
epsilons = [0.15]
results = []

for epsilon in epsilons:
    correct_original = 0
    correct_adversarial = 0
    total = 0

    for image, label in test_loader:
        image.requires_grad = True
        output_original = model(image)
        pred_original = output_original.argmax(dim=1)

        adv_image = projected_gradient_descent(model, image, eps=epsilon, eps_iter=0.01, nb_iter=40, norm=np.inf)
        output_adversarial = model(adv_image)
        pred_adversarial = output_adversarial.argmax(dim=1)

        correct_original += (pred_original == label).sum().item()
        correct_adversarial += (pred_adversarial == label).sum().item()
        total += label.size(0)

    acc_original = 100 * correct_original / total
    acc_adversarial = 100 * correct_adversarial / total
    results.append({"Epsilon": epsilon, "Accuracy_Original": acc_original, "Accuracy_Adversarial": acc_adversarial})
    print(f"Epsilon: {epsilon}")
    print(f"✅ Accuracy on original images: {acc_original:.2f}%")
    print(f"⚠️ Accuracy after PGD attack: {acc_adversarial:.2f}%")

In [ ]:
# Save results to CSV
df = pd.DataFrame(results)
df.to_csv("pgd_attack_results.csv", index=False)
print("📂 Results saved to pgd_attack_results.csv")